## PRISM Climate Data Acquisition and Processing

I am using the package and instructions from this Github: https://github.com/sdtaylor/pyPRISMClimate
to download PRISM data.

I'm collecting daily mean temperatures and daily precipitation levels, from which I am computing the Accumulated Growing Degree Days with a base temperature of 0C and an accumulation period of 100 days.

AGDD = SUM: T_mean_i - T_base; i = n-100:n

As of now (8pm Wed), I have AGDD with 100 day lag and 0 deg base temp for 2016 (lag starts ~april) - Present

I'll also sum precipitation levels similarly.

In [1]:
from pyPRISMClimate import get_prism_monthlys, get_prism_monthly_single, get_prism_dailys, get_prism_daily_single, get_prism_normals

import numpy as np
import pyproj
import rasterio
import rasterio.mask
import fiona

import os
from datetime import date, timedelta

In [5]:
#get_prism_monthlys(variable='tmean', years=[2019], months=[1,2,3,4,5,6], dest_path='prism/')

In [20]:
# Create directory structure as needed
def make_dirs (dir_path):
    dirs = dir_path.rstrip('/').split('/')
    curr_path = ''
    for d in dirs:
        curr_path+=d+'/'
        if (os.path.isdir(curr_path) == False):
            os.mkdir(curr_path)

### Data Download

In [34]:
# download daily mean temp for 2018
#years = [2014,2015,2016,2017,2018,2019,2020]
years = [2013,2012]
what = ['tmean']

for w in what:
    for y in years:
        dest_path = 'prism/daily/%s/%d/'%(w,y)
        make_dirs(dest_path)
        get_prism_dailys(w,min_date=('%d-01-01'%y), max_date=('%d-12-31'%y), dest_path=dest_path)

KeyboardInterrupt: 

### AGDD Calculation

In [2]:
class raster_dataset:
    
    def __init__(self, filename):#, window=None):
        #self.window = window
        self.src = rasterio.open(filename)
        self.src_coord = pyproj.Proj(self.src.crs)
        self.lonlat = pyproj.Proj(init='epsg:4326')
        self.data = self.src.read(1)#, window=self.window)

    def get_gps (self, row, col):
        east, north = self.src.xy(row,col) # image --> spatial coordinates
        lon,lat = pyproj.transform(self.src_coord, self.lonlat, east, north)
        value = self.data[row, col]
        return lon, lat
    
    # input: longitude, latitude (gps coordinate)
    # return: data value at input location(s)
    def get_value (self, lon, lat):
        east,north = pyproj.transform(self.lonlat, self.src_coord, lon, lat)
    
        # What is the corresponding row and column in our image?
        row, col = self.src.index(east, north) # spatial --> image coordinates
        #print(f'row,col=\t\t({row},{col})')
    
        # What is the value at that index?
        value = self.data[row, col]
        return value
    
    def get_rc (self, lon, lat):
        east,north = pyproj.transform(self.lonlat, self.src_coord, lon, lat)
        return self.src.index(east, north)
    
    def write_file (self, filename, data=None):
        
        if data is None:
            data = self.data
    
        with rasterio.Env():
        
            # Write an array as a raster band to a new 8-bit file. For
            # the new file's profile, we start with the profile of the source
            profile = self.src.profile
        
            # And then change the band count to 1, set the
            # dtype to uint8, and specify LZW compression.
            profile.update(
                dtype=rasterio.float32,
                count=1,
                compress='lzw')
        
            with rasterio.open(filename, 'w', **profile) as dst:
                dst.write(data.astype(rasterio.float32), 1)


In [9]:

def compute_avg_prism (dates, field):
    
    num_dates = len(dates)

    input_fn = 'prism/daily/'+field+'/%s/PRISM_'+field+'_%s_4kmD2_%s%02d%02d_bil.bil'
    
    raster_data = []
    
    for d in dates:
        try:
            d_data = raster_dataset(input_fn % (d.year,'stable',d.year,int(d.month),int(d.day)))
        except Exception as e:
            #print (e)
            try:
                d_data = raster_dataset(input_fn % (d.year,'provisional',d.year,int(d.month),int(d.day)))
            except:
                d_data = raster_dataset(input_fn % (d.year,'early',d.year,int(d.month),int(d.day)))
 
        raster_data.append(d_data.data)
            
    # return mean of all dates read in, plus a raster_dataset object to use to write output (doesn't matter which so just use the last one)
    return (np.mean (raster_data,axis=0), d_data)     

In [15]:
# compute the 2008-2019 historic averages
start = date(2020,5,25)
end = date(2020,12,31)
delta = end - start

for i in range (delta.days + 1):
    dates = []
    day = start + timedelta(days=i)
    yrs = 1
    while (yrs < 12):
        dates.append(day+timedelta(days=(-365*yrs-2)))
        dates.append(day+timedelta(days=(-365*yrs-1)))
        dates.append(day+timedelta(days=-365*yrs))
        dates.append(day+timedelta(days=(-365*yrs+1)))
        dates.append(day+timedelta(days=(-365*yrs+2)))
        yrs += 1
        
    for field in ['tmean','ppt']:
        result, write_raster = compute_avg_prism (dates,field)
        output_fn = ('prism/daily/'+field+'/%s/PRISM_'+field+'_%s_'+'4kmD2_%s%02d%02d_bil.bil') % (day.year,'historic',day.year,int(day.month),int(day.day))
        print (output_fn)
        write_raster.write_file(output_fn, result)
    

prism/daily/tmean/2020/PRISM_tmean_historic_4kmD2_20200525_bil.bil
prism/daily/ppt/2020/PRISM_ppt_historic_4kmD2_20200525_bil.bil
prism/daily/tmean/2020/PRISM_tmean_historic_4kmD2_20200526_bil.bil
prism/daily/ppt/2020/PRISM_ppt_historic_4kmD2_20200526_bil.bil
prism/daily/tmean/2020/PRISM_tmean_historic_4kmD2_20200527_bil.bil
prism/daily/ppt/2020/PRISM_ppt_historic_4kmD2_20200527_bil.bil
prism/daily/tmean/2020/PRISM_tmean_historic_4kmD2_20200528_bil.bil
prism/daily/ppt/2020/PRISM_ppt_historic_4kmD2_20200528_bil.bil
prism/daily/tmean/2020/PRISM_tmean_historic_4kmD2_20200529_bil.bil
prism/daily/ppt/2020/PRISM_ppt_historic_4kmD2_20200529_bil.bil
prism/daily/tmean/2020/PRISM_tmean_historic_4kmD2_20200530_bil.bil
prism/daily/ppt/2020/PRISM_ppt_historic_4kmD2_20200530_bil.bil
prism/daily/tmean/2020/PRISM_tmean_historic_4kmD2_20200531_bil.bil
prism/daily/ppt/2020/PRISM_ppt_historic_4kmD2_20200531_bil.bil
prism/daily/tmean/2020/PRISM_tmean_historic_4kmD2_20200601_bil.bil
prism/daily/ppt/2020/PR

prism/daily/ppt/2020/PRISM_ppt_historic_4kmD2_20200727_bil.bil
prism/daily/tmean/2020/PRISM_tmean_historic_4kmD2_20200728_bil.bil
prism/daily/ppt/2020/PRISM_ppt_historic_4kmD2_20200728_bil.bil
prism/daily/tmean/2020/PRISM_tmean_historic_4kmD2_20200729_bil.bil
prism/daily/ppt/2020/PRISM_ppt_historic_4kmD2_20200729_bil.bil
prism/daily/tmean/2020/PRISM_tmean_historic_4kmD2_20200730_bil.bil
prism/daily/ppt/2020/PRISM_ppt_historic_4kmD2_20200730_bil.bil
prism/daily/tmean/2020/PRISM_tmean_historic_4kmD2_20200731_bil.bil
prism/daily/ppt/2020/PRISM_ppt_historic_4kmD2_20200731_bil.bil
prism/daily/tmean/2020/PRISM_tmean_historic_4kmD2_20200801_bil.bil
prism/daily/ppt/2020/PRISM_ppt_historic_4kmD2_20200801_bil.bil
prism/daily/tmean/2020/PRISM_tmean_historic_4kmD2_20200802_bil.bil
prism/daily/ppt/2020/PRISM_ppt_historic_4kmD2_20200802_bil.bil
prism/daily/tmean/2020/PRISM_tmean_historic_4kmD2_20200803_bil.bil
prism/daily/ppt/2020/PRISM_ppt_historic_4kmD2_20200803_bil.bil
prism/daily/tmean/2020/PRIS

prism/daily/tmean/2020/PRISM_tmean_historic_4kmD2_20200929_bil.bil
prism/daily/ppt/2020/PRISM_ppt_historic_4kmD2_20200929_bil.bil
prism/daily/tmean/2020/PRISM_tmean_historic_4kmD2_20200930_bil.bil
prism/daily/ppt/2020/PRISM_ppt_historic_4kmD2_20200930_bil.bil
prism/daily/tmean/2020/PRISM_tmean_historic_4kmD2_20201001_bil.bil
prism/daily/ppt/2020/PRISM_ppt_historic_4kmD2_20201001_bil.bil
prism/daily/tmean/2020/PRISM_tmean_historic_4kmD2_20201002_bil.bil
prism/daily/ppt/2020/PRISM_ppt_historic_4kmD2_20201002_bil.bil
prism/daily/tmean/2020/PRISM_tmean_historic_4kmD2_20201003_bil.bil
prism/daily/ppt/2020/PRISM_ppt_historic_4kmD2_20201003_bil.bil
prism/daily/tmean/2020/PRISM_tmean_historic_4kmD2_20201004_bil.bil
prism/daily/ppt/2020/PRISM_ppt_historic_4kmD2_20201004_bil.bil
prism/daily/tmean/2020/PRISM_tmean_historic_4kmD2_20201005_bil.bil
prism/daily/ppt/2020/PRISM_ppt_historic_4kmD2_20201005_bil.bil
prism/daily/tmean/2020/PRISM_tmean_historic_4kmD2_20201006_bil.bil
prism/daily/ppt/2020/PR

prism/daily/tmean/2020/PRISM_tmean_historic_4kmD2_20201129_bil.bil
prism/daily/ppt/2019/PRISM_ppt_stable_4kmD2_20191201_bil.bil: No such file or directory
prism/daily/ppt/2019/PRISM_ppt_stable_4kmD2_20191202_bil.bil: No such file or directory
prism/daily/ppt/2020/PRISM_ppt_historic_4kmD2_20201129_bil.bil
prism/daily/tmean/2019/PRISM_tmean_stable_4kmD2_20191201_bil.bil: No such file or directory
prism/daily/tmean/2019/PRISM_tmean_stable_4kmD2_20191202_bil.bil: No such file or directory
prism/daily/tmean/2019/PRISM_tmean_stable_4kmD2_20191203_bil.bil: No such file or directory
prism/daily/tmean/2020/PRISM_tmean_historic_4kmD2_20201130_bil.bil
prism/daily/ppt/2019/PRISM_ppt_stable_4kmD2_20191201_bil.bil: No such file or directory
prism/daily/ppt/2019/PRISM_ppt_stable_4kmD2_20191202_bil.bil: No such file or directory
prism/daily/ppt/2019/PRISM_ppt_stable_4kmD2_20191203_bil.bil: No such file or directory
prism/daily/ppt/2020/PRISM_ppt_historic_4kmD2_20201130_bil.bil
prism/daily/tmean/2019/P

prism/daily/tmean/2020/PRISM_tmean_historic_4kmD2_20201208_bil.bil
prism/daily/ppt/2019/PRISM_ppt_stable_4kmD2_20191207_bil.bil: No such file or directory
prism/daily/ppt/2019/PRISM_ppt_stable_4kmD2_20191208_bil.bil: No such file or directory
prism/daily/ppt/2019/PRISM_ppt_stable_4kmD2_20191209_bil.bil: No such file or directory
prism/daily/ppt/2019/PRISM_ppt_stable_4kmD2_20191210_bil.bil: No such file or directory
prism/daily/ppt/2019/PRISM_ppt_stable_4kmD2_20191211_bil.bil: No such file or directory
prism/daily/ppt/2020/PRISM_ppt_historic_4kmD2_20201208_bil.bil
prism/daily/tmean/2019/PRISM_tmean_stable_4kmD2_20191208_bil.bil: No such file or directory
prism/daily/tmean/2019/PRISM_tmean_stable_4kmD2_20191209_bil.bil: No such file or directory
prism/daily/tmean/2019/PRISM_tmean_stable_4kmD2_20191210_bil.bil: No such file or directory
prism/daily/tmean/2019/PRISM_tmean_stable_4kmD2_20191211_bil.bil: No such file or directory
prism/daily/tmean/2019/PRISM_tmean_stable_4kmD2_20191212_bil.b

prism/daily/tmean/2020/PRISM_tmean_historic_4kmD2_20201216_bil.bil
prism/daily/ppt/2019/PRISM_ppt_stable_4kmD2_20191215_bil.bil: No such file or directory
prism/daily/ppt/2019/PRISM_ppt_stable_4kmD2_20191216_bil.bil: No such file or directory
prism/daily/ppt/2019/PRISM_ppt_stable_4kmD2_20191217_bil.bil: No such file or directory
prism/daily/ppt/2019/PRISM_ppt_stable_4kmD2_20191218_bil.bil: No such file or directory
prism/daily/ppt/2019/PRISM_ppt_stable_4kmD2_20191219_bil.bil: No such file or directory
prism/daily/ppt/2020/PRISM_ppt_historic_4kmD2_20201216_bil.bil
prism/daily/tmean/2019/PRISM_tmean_stable_4kmD2_20191216_bil.bil: No such file or directory
prism/daily/tmean/2019/PRISM_tmean_stable_4kmD2_20191217_bil.bil: No such file or directory
prism/daily/tmean/2019/PRISM_tmean_stable_4kmD2_20191218_bil.bil: No such file or directory
prism/daily/tmean/2019/PRISM_tmean_stable_4kmD2_20191219_bil.bil: No such file or directory
prism/daily/tmean/2019/PRISM_tmean_stable_4kmD2_20191220_bil.b

prism/daily/tmean/2020/PRISM_tmean_historic_4kmD2_20201224_bil.bil
prism/daily/ppt/2019/PRISM_ppt_stable_4kmD2_20191223_bil.bil: No such file or directory
prism/daily/ppt/2019/PRISM_ppt_stable_4kmD2_20191224_bil.bil: No such file or directory
prism/daily/ppt/2019/PRISM_ppt_stable_4kmD2_20191225_bil.bil: No such file or directory
prism/daily/ppt/2019/PRISM_ppt_stable_4kmD2_20191226_bil.bil: No such file or directory
prism/daily/ppt/2019/PRISM_ppt_stable_4kmD2_20191227_bil.bil: No such file or directory
prism/daily/ppt/2020/PRISM_ppt_historic_4kmD2_20201224_bil.bil
prism/daily/tmean/2019/PRISM_tmean_stable_4kmD2_20191224_bil.bil: No such file or directory
prism/daily/tmean/2019/PRISM_tmean_stable_4kmD2_20191225_bil.bil: No such file or directory
prism/daily/tmean/2019/PRISM_tmean_stable_4kmD2_20191226_bil.bil: No such file or directory
prism/daily/tmean/2019/PRISM_tmean_stable_4kmD2_20191227_bil.bil: No such file or directory
prism/daily/tmean/2019/PRISM_tmean_stable_4kmD2_20191228_bil.b

In [6]:
# test file reading
raster_dataset ('prism/daily/tmean/2013/PRISM_tmean_stable_4kmD2_20131102_bil.bil').data

array([[-9999., -9999., -9999., ..., -9999., -9999., -9999.],
       [-9999., -9999., -9999., ..., -9999., -9999., -9999.],
       [-9999., -9999., -9999., ..., -9999., -9999., -9999.],
       ...,
       [-9999., -9999., -9999., ..., -9999., -9999., -9999.],
       [-9999., -9999., -9999., ..., -9999., -9999., -9999.],
       [-9999., -9999., -9999., ..., -9999., -9999., -9999.]],
      dtype=float32)

In [21]:
# read all raster files from 2017
start = date(2020, 1, 1)
end = date(2020, 12, 31)
delta = end - start       # as timedelta

dates = []

count = 0
for i in range(delta.days + 1):
    day = start + timedelta(days=i)
    dates.append(day)
    count += 1
    #dates.append('%s%02d%02d'%(day.year,int(day.month),int(day.day)))
 

In [39]:
import warnings
warnings.filterwarnings('ignore')

In [22]:
# Now, do the following process in batches of 150, incrementing by 50 each time.
num_dates = len(dates)

start = 0
end = start + 160

output_fn = 'prism/daily/tmean/%s/PRISM_tmean_%s_4kmD2_%s%02d%02d_bil.bil'

while (True):
    
    if (end == num_dates):
        print ("Processing batch: %s to %s" % (str(dates[start]),str(dates[end-1])))
    else:
        print ("Processing batch: %s to %s" % (str(dates[start]),str(dates[end])))
   
    raster_data = []
    
    dates_batch = dates[start:end]
    #count = 0
    for d in dates_batch:
        count += 1
        #print ('prism/daily/tmean/%s/PRISM_tmean_stable_4kmD2_%s%02d%02d_bil.bil' % (d.year,d.year,int(d.month),int(d.day)))
        try:
            d_data = raster_dataset(output_fn % (d.year,'stable',d.year,int(d.month),int(d.day)))
        except:
            try:
                d_data = raster_dataset(output_fn % (d.year,'provisional',d.year,int(d.month),int(d.day)))
            except:
                try:
                    d_data = raster_dataset(output_fn % (d.year,'early',d.year,int(d.month),int(d.day)))
                    # this means that the dataset is not available, so gather all prior years and accumulate them
                except:
                    d_data = raster_dataset(output_fn % (d.year,'historic',d.year,int(d.month),int(d.day)))
                
        # for our purposes, we can set -9999 values to zero (will make addition easier)
        raster_data.append(d_data)
        
    # subtract base temp
    
    #print (count)
    
    # 32 degree AGDD = 0 deg C, so just cumulative sum of values
    lag_period = 100 # 10-day AGDD
    base_temp = 0
    
    agdd = np.zeros((len(raster_data)-lag_period,raster_data[0].data.shape[0],raster_data[0].data.shape[1]))
    
    for i in range(lag_period):
        agdd[0] += raster_data[i].data
    
    for i in range(lag_period+1,len(raster_data)):
        agdd[i-lag_period] = agdd[i-lag_period-1]+raster_data[i].data-raster_data[i-(lag_period+1)].data
    
    dest_path = 'prism/daily/agdd0/%s/'
    for y in range(2008,2021):
        make_dirs(dest_path%str(y))
    dest_path += 'PRISM_agdd0_%s%02d%02d.tif'
        
    # write agdd files
    for i in range(len(raster_data)-lag_period):
        d = dates_batch[i+lag_period]
        raster_data[i+lag_period].write_file(dest_path % (d.year,d.year,int(d.month),int(d.day)), data=agdd[i])
     
    # we are done when we've gone through all specified dates
    if (end == num_dates):
        break
        
    start += 50
    end += 50
    
    # we want to stop at the last specified date
    if (end > num_dates):
        end = num_dates
        start = num_dates - 160

        

Processing batch: 2020-01-01 to 2020-06-09
Processing batch: 2020-02-20 to 2020-07-29
Processing batch: 2020-04-10 to 2020-09-17
Processing batch: 2020-05-30 to 2020-11-06
Processing batch: 2020-07-19 to 2020-12-26
Processing batch: 2020-07-25 to 2020-12-31


### PPT Calculation

In [23]:
# generate date range
start = date(2020, 1, 1)
end = date(2020, 12, 31)
delta = end - start       # as timedelta

dates = []

for i in range(delta.days + 1):
    day = start + timedelta(days=i)
    dates.append(day)
    count += 1
    #dates.append('%s%02d%02d'%(day.year,int(day.month),int(day.day)))
 

In [24]:
# Now, do the following process in batches of 150, incrementing by 50 each time.
num_dates = len(dates)

start = 0
end = start + 160

output_fn = 'prism/daily/ppt/%s/PRISM_ppt_%s_4kmD2_%s%02d%02d_bil.bil'

while (True):
    
    if (end == num_dates):
        print ("Processing batch: %s to %s" % (str(dates[start]),str(dates[end-1])))
    else:
        print ("Processing batch: %s to %s" % (str(dates[start]),str(dates[end])))
   
    raster_data = []
    
    dates_batch = dates[start:end]
    #count = 0
    for d in dates_batch:
        count += 1
        #print ('prism/daily/tmean/%s/PRISM_tmean_stable_4kmD2_%s%02d%02d_bil.bil' % (d.year,d.year,int(d.month),int(d.day)))
        try:
            d_data = raster_dataset(output_fn % (d.year,'stable',d.year,int(d.month),int(d.day)))
        except:
            try:
                d_data = raster_dataset(output_fn % (d.year,'provisional',d.year,int(d.month),int(d.day)))
            except:
                try:
                    d_data = raster_dataset(output_fn % (d.year,'early',d.year,int(d.month),int(d.day)))
                    # this means that the dataset is not available, so gather all prior years and accumulate them
                except:
                    d_data = raster_dataset(output_fn % (d.year,'historic',d.year,int(d.month),int(d.day)))
           
        # for our purposes, we can set -9999 values to zero (will make addition easier)
        raster_data.append(d_data)
            
    #print (count)
    
    # cumulative sum of values
    lag_period = 100 # number of previous days to sum
    
    cum_ppt = np.zeros((len(raster_data)-lag_period,raster_data[0].data.shape[0],raster_data[0].data.shape[1]))
    
    for i in range(lag_period):
        cum_ppt[0] += raster_data[i].data
    
    for i in range(lag_period+1,len(raster_data)):
        cum_ppt[i-lag_period] = cum_ppt[i-lag_period-1]+raster_data[i].data-raster_data[i-(lag_period+1)].data
    
    dest_path = 'prism/daily/cum_ppt_100/%s/'
    for y in range(2008,2021):
        make_dirs(dest_path%str(y))
    dest_path += 'PRISM_cum_ppt_100_%s%02d%02d.tif'
        
    # write cum_ppt files
    for i in range(len(raster_data)-lag_period):
        d = dates_batch[i+lag_period]
        raster_data[i+lag_period].write_file(dest_path % (d.year,d.year,int(d.month),int(d.day)), data=cum_ppt[i])
     
    # we are done when we've gone through all specified dates
    if (end == num_dates):
        break
    
    start += 50
    end += 50
    
    # we want to stop at the last specified date
    if (end > num_dates):
        end = num_dates
        start = num_dates - 160


Processing batch: 2020-01-01 to 2020-06-09
Processing batch: 2020-02-20 to 2020-07-29
Processing batch: 2020-04-10 to 2020-09-17
Processing batch: 2020-05-30 to 2020-11-06
Processing batch: 2020-07-19 to 2020-12-26
Processing batch: 2020-07-25 to 2020-12-31


In [44]:
# test
raster_dataset(dest_path %('2008','2008',5,1)).get_value(-116.989241,32.697177)

90.349

In [201]:
#raster_data[10].write_file('test_file',data=agdd[0])
raster_dataset('').get_value(-116.989241,32.697177)
#raster_dataset('prism/daily/tmean/2017/PRISM_tmean_stable_4kmD2_20170115_bil.bil').get_value(-116.989241,32.697177)

#raster_data[10].src.profile

136.658

In [44]:
#elevation_filename = lambda a: 'srtm/srtm_%s/srtm_%s.tif'%(a,a)
src = rasterio.open('prism/PRISM_tmax_stable_4kmD2_20170101_bil.bil')
src_coord = pyproj.Proj(src.crs)
lonlat = pyproj.Proj(init='epsg:4326')
data = src.read(1)
#data.shape
get_value(-122.697995,38.292092) # petaluma
get_value(-116.989241,32.697177) # san diego
get_value(-118.866132,37.501283) # high sierra

-3.982

In [100]:
#Window(col_off, row_off, width, height)
#rasterio.open('prism/daily/tmean/2017/PRISM_tmean_stable_4kmD2_20170101_bil.bil').read(1,window=rasterio.windows.Window(187,-5,233,273))



array([[-9.9990000e+03, -9.9990000e+03, -9.9990000e+03, ...,
        -9.9990000e+03, -9.9990000e+03, -9.9990000e+03],
       [-9.9990000e+03, -9.9990000e+03, -9.9990000e+03, ...,
        -9.9990000e+03, -9.9990000e+03, -9.9990000e+03],
       [-9.9990000e+03, -9.9990000e+03, -9.9990000e+03, ...,
        -9.9990000e+03, -9.9990000e+03, -9.9990000e+03],
       ...,
       [-3.6710002e+00, -3.8730001e+00, -3.7920003e+00, ...,
         5.9300005e-01, -3.6800003e-01, -3.7800002e-01],
       [-2.8610001e+00, -3.8430002e+00, -3.7720001e+00, ...,
        -1.8400002e-01,  8.8000007e-02, -5.7000005e-01],
       [-2.0370002e+00, -3.6390002e+00, -3.7730002e+00, ...,
         6.8000004e-02, -4.2300001e-01, -3.1330001e+00]], dtype=float32)

In [107]:
# Here we will start building accumulated growing degree days
win = rasterio.windows.Window(187,-5,233,273)
raster1 = raster_dataset('prism/daily/tmean/2017/PRISM_tmean_stable_4kmD2_20170101_bil.bil',window=win)
#raster2 = raster_dataset('prism/daily/tmean/2018/PRISM_tmean_stable_4kmD2_20180801_bil.bil')

In [144]:
with rasterio.open('prism/daily/tmean/2017/PRISM_tmean_stable_4kmD2_20170101_bil.bil') as src:
    #window = Window(col_off=13, row_off=3, width=757, height=711)
    window = rasterio.windows.Window(col_off=187, row_off=-5, width=233, height=273)
    kwargs = src.meta.copy()
    kwargs.update({
        'height': window.height,
        'width': window.width,
        'transform': rasterio.windows.transform(window, src.transform)})
    print (kwargs)
    with rasterio.open('ca_subset.tif', 'w', **kwargs) as dst:
        dst.write(src.read(window=window))


{'driver': 'EHdr', 'dtype': 'float32', 'nodata': -9999.0, 'width': 233, 'height': 273, 'count': 1, 'crs': CRS.from_dict(init='epsg:4269'), 'transform': Affine(0.041666666667, 0.0, -117.2291666666045,
       0.0, -0.041666666667, 50.1458333333345)}


In [145]:
raster1ca = raster_dataset('ca_subset.tif')
raster1 = raster_dataset('prism/daily/tmean/2017/PRISM_tmean_stable_4kmD2_20170101_bil.bil')
#print (raster1.get_rc(-116.989241,32.697177)) # san diego
print (raster1.get_rc(-125.2,32.41)) # san diego
print (raster1.get_rc(-119.2,38.41)) # san diego
print (raster1.get_rc(-113.85,42.14)) # san diego


print (raster1ca.get_value(-116.989241,32.697177)) # san diego

(420, -5)
(276, 139)
(187, 268)


IndexError: index 418 is out of bounds for axis 0 with size 273

In [124]:
#ca = gpd.read_file('ca-state-boundary/CA_State_TIGER2016.shp')

with fiona.open('ca-state-boundary/CA_State_TIGER2016.shp', "r") as shapefile:
    shapes = [feature["geometry"] for feature in shapefile]
    
with rasterio.open('prism/daily/tmean/2017/PRISM_tmean_stable_4kmD2_20170101_bil.bil') as src:
    out_image, out_transform = rasterio.mask.mask(src, shapes, crop=True)
    out_meta = src.meta

ValueError: Input shapes do not overlap raster.

In [109]:
#raster1.get_value(-116.989241,32.697177)

raster1.src.window_transform(win)

Affine(0.041666666667, 0.0, -117.2291666666045,
       0.0, -0.041666666667, 50.1458333333345)

In [92]:
#raster.get_value(-116.989241,32.697177)
print (raster1.get_gps(620,1404))
print (raster2.get_gps(620,1404))
#print (raster.get_gps(0,0))
print (raster1.get_rc(-125.19,32.41))
print (raster1.get_rc(-113.85,42.14))
#raster.data.shape

(-66.4999999999532, 24.083333333312403)
(-66.4999999999532, 24.083333333312403)
(420, -5)
(187, 268)
